<a href="https://colab.research.google.com/github/kd303/trnsfrmr_pytrch_end_p1/blob/main/session4-rnn_hands_on/END2_Session_4_IMDB_Sentiment_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [117]:
import torch
from torchtext.legacy import data

SEED = 1234

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

## If the Include length is not true, we dont get the length so at the time of padding 
## length becomes important, so change it.

TEXT = data.Field(tokenize = 'spacy',
                  tokenizer_language = 'en_core_web_sm',batch_first =True, include_lengths=True)
LABEL = data.LabelField(dtype = torch.float, batch_first =True,is_target=True)

In [141]:
from torchtext.legacy import datasets

train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)

In [142]:
print(f'Number of training examples: {len(train_data)}')
print(f'Number of testing examples: {len(test_data)}')

Number of training examples: 25000
Number of testing examples: 25000


In [119]:
## vars returns the changeable attribute of __dict__ from the object
print(vars(train_data.examples[0]))

{'text': ['This', 'review', 'may', 'contain', 'some', 'spoilers.<br', '/><br', '/>The', 'remake', 'of', 'the', 'classic', '1974', 'car', 'chase', 'movie', 'Gone', 'in', '60', 'Seconds', 'begins', 'well', '.', 'Actually', 'it', 'is', 'well', 'acted', 'and', 'the', 'plot', 'moves', 'quite', 'well', '.', 'But', 'even', 'a', 'big', 'Hollywood', 'budget', 'does', "n't", 'change', 'the', 'fact', 'that', 'the', 'original', 'plot', 'was', 'more', 'believable', '.', 'For', 'those', 'who', 'do', "n't", 'know', ',', 'the', 'original', 'plot', 'had', 'the', 'thieves', 'working', 'as', 'insurance', 'inspectors', '.', 'Who', 'would', 'suspect', 'them', '.', 'But', 'even', 'with', 'a', 'change', 'to', 'nearly', 'every', 'aspect', 'of', 'H.B.', 'Halicki', "'s", 'original', ',', 'the', 'remake', 'is', 'a', 'very', 'good', 'movie', ',', 'until', 'we', 'get', 'to', 'the', 'final', 'chase', 'scene', ',', 'the', 'part', 'of', 'the', '74', 'version', 'that', 'made', 'it', 'great', '.', 'The', 'one', 'in', '

In [143]:
import random

train_data, valid_data = train_data.split(random_state = random.seed(SEED))

In [144]:
print(f'Number of training examples: {len(train_data)}')
print(f'Number of validation examples: {len(valid_data)}')
print(f'Number of testing examples: {len(test_data)}')

Number of training examples: 17500
Number of validation examples: 7500
Number of testing examples: 25000


In [145]:
MAX_VOCAB_SIZE = 25_000

TEXT.build_vocab(train_data, max_size = MAX_VOCAB_SIZE)
LABEL.build_vocab(train_data)

In [146]:
print(f"Unique tokens in TEXT vocabulary: {len(TEXT.vocab)}")
print(f"Unique tokens in LABEL vocabulary: {len(LABEL.vocab)}")

Unique tokens in TEXT vocabulary: 25002
Unique tokens in LABEL vocabulary: 2


In [147]:
print(TEXT.vocab.freqs.most_common(20))

[('the', 203883), (',', 193363), ('.', 166762), ('and', 110063), ('a', 109530), ('of', 100924), ('to', 93842), ('is', 76734), ('in', 61619), ('I', 54378), ('it', 53503), ('that', 49231), ('"', 44186), ("'s", 43472), ('this', 42441), ('-', 37108), ('/><br', 35778), ('was', 35097), ('as', 30575), ('with', 30303)]


In [148]:
print(TEXT.vocab.itos[:10])

['<unk>', '<pad>', 'the', ',', '.', 'and', 'a', 'of', 'to', 'is']


In [149]:
print(LABEL.vocab.stoi)

defaultdict(None, {'neg': 0, 'pos': 1})


In [151]:
BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    sort_key = lambda x: len(x.text),
    sort_within_batch=True,
    batch_size = BATCH_SIZE,
    device = device)

In [232]:
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim, num_layers, dropout):
        
        super().__init__()
        
        self.embedding = nn.Embedding(input_dim, embedding_dim)
        
        self.rnn = nn.RNN(embedding_dim, hidden_dim)
        # Batch first - (batch, seq, feature)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers = num_layers, 
                          dropout = dropout, batch_first=True)

        self.GRU = nn.GRU(embedding_dim, hidden_dim, num_layers = num_layers,
                         batch_first=True, dropout = dropout)
        self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, text, text_length):

        #text = [sent len, batch size]
        
        
        embedded = self.embedding(text)
        
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_length.cpu(), batch_first = True)
        #embedded = [sent len, batch size, emb dim]
        
        # output, hidden = self.rnn(embedded)
        output, (hidden, cell)  = self.lstm(packed_embedded)
        # print("hidden", hidden.shape, "output", output.shape, "cell", cell.shape)
        
        #output = [sent len, batch size, hid dim]
        #hidden = [1, batch size, hid dim]
        
        # assert torch.equal(output[-1,:,:], hidden.squeeze(0))
        return self.fc(hidden)[0]
        # return self.fc(hidden.squeeze(0))

In [233]:

INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = 1
NUM_LAYERS = 2
dropout = 0.2
model = RNN(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM, NUM_LAYERS, dropout)

In [234]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The LSTEM model has {count_parameters(model):,} trainable parameters')

The LSTEM model has 4,154,729 trainable parameters


In [218]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The GRU model has {count_parameters(model):,} trainable parameters')

The GRU model has 4,154,729 trainable parameters


In [219]:
import torch.optim as optim

optimizer = optim.SGD(model.parameters(), lr=1e-3)

In [220]:
criterion = nn.BCEWithLogitsLoss()

In [237]:
model = model.to(device)
criterion = criterion.to(device)

In [158]:
device

device(type='cuda')

In [194]:
print(model)

RNN(
  (embedding): Embedding(25002, 100)
  (rnn): RNN(100, 256)
  (lstm): LSTM(100, 256, num_layers=2, batch_first=True, dropout=0.2)
  (fc): Linear(in_features=256, out_features=1, bias=True)
)


In [195]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """
    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [235]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()

        text, text_length = batch.text

        # LSTM        
        predictions = model(text, text_length).squeeze(1)
        # GRU
        # predictions = model(text, text_length)
        
        # print(predictions.shape, batch.label.shape)
        
        loss = criterion(predictions, batch.label)
        
        acc = binary_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [227]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            text, text_length = batch.text
            predictions = model(text,text_length).squeeze()
            
            loss = criterion(predictions, batch.label)

            # print(predictions.shape, batch.label.shape)
            
            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [204]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [208]:
N_EPOCHS = 5

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 46s
	Train Loss: 0.694 | Train Acc: 49.46%
	 Val. Loss: 0.694 |  Val. Acc: 50.48%
Epoch: 02 | Epoch Time: 0m 45s
	Train Loss: 0.694 | Train Acc: 49.45%
	 Val. Loss: 0.694 |  Val. Acc: 50.48%
Epoch: 03 | Epoch Time: 0m 46s
	Train Loss: 0.694 | Train Acc: 49.45%
	 Val. Loss: 0.694 |  Val. Acc: 50.48%
Epoch: 04 | Epoch Time: 0m 45s
	Train Loss: 0.694 | Train Acc: 49.44%
	 Val. Loss: 0.694 |  Val. Acc: 50.48%
Epoch: 05 | Epoch Time: 0m 45s
	Train Loss: 0.694 | Train Acc: 49.45%
	 Val. Loss: 0.694 |  Val. Acc: 50.48%


In [238]:
N_EPOCHS = 5

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 45s
	Train Loss: 0.693 | Train Acc: 50.31%
	 Val. Loss: 0.694 |  Val. Acc: 49.35%


KeyboardInterrupt: ignored

In [209]:
model.load_state_dict(torch.load('tut1-model.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.694 | Test Acc: 49.51%
